In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# --- Step 1: Read in the Data Files ---
# Update file paths as needed for your local environment.
sales_df = pd.read_csv('./m5-forecasting-accuracy/sales_train_validation.csv')
calendar_df = pd.read_csv('./m5-forecasting-accuracy/calendar.csv')
sell_prices_df = pd.read_csv('./m5-forecasting-accuracy/sell_prices.csv')

In [ ]:
# --- Step 2: Convert Sales Data from Wide to Long Format ---
# The sales data is stored with one row per item and columns 'd_1' to 'd_N' representing dates.
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
value_vars = [col for col in sales_df.columns if col.startswith('d_')]
sales_long = pd.melt(sales_df, id_vars=id_vars, value_vars=value_vars,
                     var_name='d', value_name='sales')

In [ ]:
# --- Step 3: Merge with Calendar Data ---
# The calendar file has a column 'd' that aligns with our melted sales data.
sales_long = sales_long.merge(calendar_df, on='d', how='left')

In [ ]:
# --- Step 4: Merge with Sell Prices ---
# Merge pricing information based on store, item, and the week (wm_yr_wk).
sales_long = sales_long.merge(sell_prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [ ]:
# --- Step 5: Preprocess Dates and Sort Data ---
# Convert the 'date' column to datetime and sort by 'id' then date.
sales_long['date'] = pd.to_datetime(sales_long['date'])
sales_long.sort_values(by=['id', 'date'], inplace=True)

In [ ]:
# --- Step 6: Create Lag Features ---
# These features capture previous days' sales, e.g., 7-day and 28-day lags.
lag_days = [7, 28]
for lag in lag_days:
    sales_long[f'sales_lag_{lag}'] = sales_long.groupby('id')['sales'].shift(lag)

In [ ]:
# --- Step 7: Create Rolling Statistics ---
# Calculate rolling mean and standard deviation for given window sizes.
rolling_windows = [7, 28]
for window in rolling_windows:
    # Use shift(1) to avoid using current day's sales in the rolling calculation.
    sales_long[f'rolling_mean_{window}'] = sales_long.groupby('id')['sales'].transform(
        lambda x: x.shift(1).rolling(window).mean())
    sales_long[f'rolling_std_{window}'] = sales_long.groupby('id')['sales'].transform(
        lambda x: x.shift(1).rolling(window).std())

In [ ]:
# --- Step 8: Generate Calendar-Based Features ---
# Extract date parts that can capture seasonality: day of week, month, year, and weekend indicator.
sales_long['dayofweek'] = sales_long['date'].dt.dayofweek
sales_long['month'] = sales_long['date'].dt.month
sales_long['year'] = sales_long['date'].dt.year
sales_long['is_weekend'] = sales_long['dayofweek'].isin([5, 6]).astype(int)

In [ ]:
# Optionally, flag special events using the calendar data (e.g., holidays or events)
# For example, you could create a binary feature from an event column:
if 'event_name_1' in sales_long.columns:
    sales_long['is_event'] = sales_long['event_name_1'].notnull().astype(int)

In [ ]:
# --- Step 9: Create Price-Based and Interaction Features ---
# Example: Calculate the price change rate over time for each item.
sales_long['price_change_rate'] = sales_long.groupby('id')['sell_price'].pct_change()

C:\Users\jakey\AppData\Local\Temp\ipykernel_6864\1666045511.py:3: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  sales_long['price_change_rate'] = sales_long.groupby('id')['sell_price'].pct_change()


In [ ]:
# You could also create an interaction feature between promotional events and price
sales_long['price_event_interaction'] = sales_long['sell_price'] * sales_long.get('is_event', 0)

In [ ]:
# Preview the enriched dataset
print(sales_long.head())

                                 id      item_id  dept_id cat_id store_id  \
1612    FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
32102   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
62592   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
93082   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
123572  FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   

       state_id    d  sales       date  wm_yr_wk  ... sales_lag_28  \
1612         CA  d_1      3 2011-01-29     11101  ...          NaN   
32102        CA  d_2      0 2011-01-30     11101  ...          NaN   
62592        CA  d_3      0 2011-01-31     11101  ...          NaN   
93082        CA  d_4      1 2011-02-01     11101  ...          NaN   
123572       CA  d_5      4 2011-02-02     11101  ...          NaN   

        rolling_mean_7  rolling_std_7  rolling_mean_28 rolling_std_28  \
1612               NaN            NaN      

In [ ]:
sales_long.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sales_lag_7',
       'sales_lag_28', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_28',
       'rolling_std_28', 'dayofweek', 'is_weekend', 'is_event',
       'price_change_rate', 'price_event_interaction'],
      dtype='object')

In [ ]:
# !pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.5 MB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.4/1.5 MB 2.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 2.7 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 2.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.7 MB/s eta 0:00:01
   ------------------------- -------------- 0.9/1.5 MB 2.8 MB/s eta 0:00:01
   ---------------------------------- ----- 1.2/1.5 MB 3.0 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.5 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.0 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [ ]:
# Remove initial rows with NaN due to lag/rolling calculations
df_model = sales_long.dropna(subset=['sales_lag_28'])

# For this example, we choose a date cutoff.
# Adjust the date thresholds based on your available data.
train_df = df_model[df_model['date'] < '2016-04-25']
valid_df = df_model[(df_model['date'] >= '2016-04-25') & (df_model['date'] <= '2016-05-22')]

# Define feature columns (you can expand this list as you refine your features)
features = [col for col in df_model.columns if col.startswith('sales_lag') or
            col.startswith('rolling_') or col in ['dayofweek', 'month', 'is_weekend', 'price_change_rate']]
target = 'sales'

# Split into X and y
X_train = train_df[features]
y_train = train_df[target]
X_valid = valid_df[features]
y_valid = valid_df[target]

In [ ]:
# Debug prints: Verify that the data is non-empty and correctly shaped.
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (57473650, 10)
y_train shape: (57473650,)
X_valid shape: (0, 10)
y_valid shape: (0,)


In [ ]:
# Debug: Print available date range in df_model
print("Min date in df_model:", df_model['date'].min())
print("Max date in df_model:", df_model['date'].max())

# For example, you might set the validation period as the last 28 days of available data.
max_date = df_model['date'].max()
train_df = df_model[df_model['date'] < (max_date - pd.Timedelta(days=28))]
valid_df = df_model[df_model['date'] >= (max_date - pd.Timedelta(days=28))]

# Confirm that both training and validation sets are non-empty.
print("Train set date range:", train_df['date'].min(), "-", train_df['date'].max())
print("Validation set date range:", valid_df['date'].min(), "-", valid_df['date'].max())

print("X_train shape:", train_df[features].shape)
print("y_train shape:", train_df[target].shape)
print("X_valid shape:", valid_df[features].shape)
print("y_valid shape:", valid_df[target].shape)

Min date in df_model: 2011-02-26 00:00:00
Max date in df_model: 2016-04-24 00:00:00
Train set date range: 2011-02-26 00:00:00 - 2016-03-26 00:00:00
Validation set date range: 2016-03-27 00:00:00 - 2016-04-24 00:00:00
X_train shape: (56589440, 10)
y_train shape: (56589440,)
X_valid shape: (884210, 10)
y_valid shape: (884210,)


In [ ]:
# Create LightGBM Datasets from your training and validation data.
lgb_train = lgb.Dataset(train_df[features], label=train_df[target])
lgb_valid = lgb.Dataset(valid_df[features], label=valid_df[target])

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1  # to suppress internal logging
}

# Train the LightGBM model using callbacks for early stopping and logging.
model = lgb.train(
    params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_valid],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

print("Best iteration:", model.best_iteration)

Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.14317	valid_1's rmse: 1.99707
[100]	training's rmse: 2.10865	valid_1's rmse: 1.97258
[150]	training's rmse: 2.09963	valid_1's rmse: 1.96996
[200]	training's rmse: 2.09376	valid_1's rmse: 1.96894
[250]	training's rmse: 2.08948	valid_1's rmse: 1.96838
[300]	training's rmse: 2.0858	valid_1's rmse: 1.96775
[350]	training's rmse: 2.0828	valid_1's rmse: 1.96757
[400]	training's rmse: 2.08022	valid_1's rmse: 1.96725
[450]	training's rmse: 2.0778	valid_1's rmse: 1.967
Early stopping, best iteration is:
[441]	training's rmse: 2.07824	valid_1's rmse: 1.96694
Best iteration: 441


In [ ]:
# Save the model to a file.
model.save_model('m5_trained_model.txt')

In [ ]:
# Extract feature importances.
importances = model.feature_importance()
feature_names = features  # The feature list used during training.

# Combine and display in a DataFrame
feat_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feat_imp)

             feature  importance
3     rolling_mean_7        2874
5    rolling_mean_28        2007
2       sales_lag_28        1887
1        sales_lag_7        1812
7          dayofweek        1503
0              month        1400
4      rolling_std_7         979
6     rolling_std_28         419
9  price_change_rate         320
8         is_weekend          29


In [ ]:
# Choose a sample time series 'id'
sample_id = df_model['id'].unique()[0]
sample_series = df_model[df_model['id'] == sample_id].copy().sort_values('date')
last_known_date = sample_series['date'].max()
forecast_dates = pd.date_range(start=last_known_date + pd.Timedelta(days=1), periods=28)

# Create a temporary DataFrame for recursive forecasting.
temp_series = sample_series.copy()
forecast_preds = []

for forecast_date in forecast_dates:
    new_features = {}
    new_features['date'] = forecast_date

    # Generate lag features for forecast_date.
    for lag in [7, 28]:
        lag_date = forecast_date - pd.Timedelta(days=lag)
        lag_value = temp_series.loc[temp_series['date'] == lag_date, 'sales']
        new_features[f'sales_lag_{lag}'] = lag_value.values[0] if not lag_value.empty else np.nan

    # Compute rolling statistics.
    for window in [7, 28]:
        window_start = forecast_date - pd.Timedelta(days=window)
        window_end = forecast_date - pd.Timedelta(days=1)
        window_sales = temp_series[(temp_series['date'] >= window_start) & (temp_series['date'] <= window_end)]['sales']
        new_features[f'rolling_mean_{window}'] = window_sales.mean() if len(window_sales) > 0 else np.nan
        new_features[f'rolling_std_{window}'] = window_sales.std() if len(window_sales) > 0 else np.nan

    # Calendar features.
    new_features['dayofweek'] = forecast_date.dayofweek
    new_features['month'] = forecast_date.month
    new_features['is_weekend'] = int(forecast_date.dayofweek in [5, 6])

    # Set default for price_change_rate (customize as needed).
    new_features['price_change_rate'] = 0

    # Create a DataFrame row for prediction.
    new_row_df = pd.DataFrame([new_features])
    X_new = new_row_df[features]
    pred_sales = model.predict(X_new)[0]
    forecast_preds.append(pred_sales)

    # Append the new forecast to update features recursively.
    new_features['sales'] = pred_sales
    new_features['id'] = sample_id
    temp_series = pd.concat([temp_series, pd.DataFrame([new_features])], ignore_index=True)

# Compile forecasts for display.
forecast_df = pd.DataFrame({
    'date': forecast_dates,
    'predicted_sales': forecast_preds
})
print("Forecasted Sales for id:", sample_id)
print(forecast_df)

Forecasted Sales for id: FOODS_1_001_CA_1_validation
         date  predicted_sales
0  2016-04-25         1.108243
1  2016-04-26         0.950177
2  2016-04-27         0.950177
3  2016-04-28         0.966403
4  2016-04-29         1.134161
5  2016-04-30         1.169028
6  2016-05-01         1.198295
7  2016-05-02         1.111982
8  2016-05-03         0.900066
9  2016-05-04         0.933213
10 2016-05-05         0.853214
11 2016-05-06         0.930345
12 2016-05-07         1.092960
13 2016-05-08         1.148746
14 2016-05-09         0.940508
15 2016-05-10         0.850561
16 2016-05-11         0.846066
17 2016-05-12         0.846066
18 2016-05-13         0.903890
19 2016-05-14         1.114828
20 2016-05-15         1.052194
21 2016-05-16         1.001207
22 2016-05-17         0.775999
23 2016-05-18         0.775999
24 2016-05-19         0.705013
25 2016-05-20         0.755930
26 2016-05-21         0.882916
27 2016-05-22         0.882916


In [ ]:
# Define parameters for the Tweedie model.
params_tweedie = {
    'objective': 'tweedie',
    'tweedie_variance_power': 1.1,  # Adjust between 1.0 (Poisson) and 2.0 (Gamma) as needed.
    'metric': 'rmse',               # We use RMSE here for monitoring (LightGBM does not natively evaluate WRMSSE).
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}

# Create LightGBM Datasets from the training and validation sets.
lgb_train = lgb.Dataset(train_df[features], label=train_df[target])
lgb_valid = lgb.Dataset(valid_df[features], label=valid_df[target])

# Train the model using callbacks for early stopping and logging.
model_tweedie = lgb.train(
    params_tweedie,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_valid],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

print("Best iteration for Tweedie model:", model_tweedie.best_iteration)

Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.17529	valid_1's rmse: 2.00651
[100]	training's rmse: 2.12749	valid_1's rmse: 1.97321
[150]	training's rmse: 2.11714	valid_1's rmse: 1.9695
[200]	training's rmse: 2.11025	valid_1's rmse: 1.96779
[250]	training's rmse: 2.1056	valid_1's rmse: 1.96693
[300]	training's rmse: 2.10187	valid_1's rmse: 1.96633
[350]	training's rmse: 2.09878	valid_1's rmse: 1.96592
[400]	training's rmse: 2.09576	valid_1's rmse: 1.96564
[450]	training's rmse: 2.09341	valid_1's rmse: 1.96556
[500]	training's rmse: 2.09133	valid_1's rmse: 1.9655
[550]	training's rmse: 2.08912	valid_1's rmse: 1.96537
[600]	training's rmse: 2.08702	valid_1's rmse: 1.96522
[650]	training's rmse: 2.08544	valid_1's rmse: 1.96516
Early stopping, best iteration is:
[647]	training's rmse: 2.08558	valid_1's rmse: 1.96514
Best iteration for Tweedie model: 647


In [ ]:
# Get predictions from the Tweedie model on the validation set.
val_preds_tweedie = model_tweedie.predict(valid_df[features])

# Attach the predictions to the validation DataFrame.
valid_df_with_preds = valid_df.copy()
valid_df_with_preds['predicted'] = val_preds_tweedie

In [ ]:
def simple_wrmsse(train_df, valid_df):
    """
    Computes a simplified WRMSSE score.

    Parameters:
        train_df : DataFrame
            Training data including 'id', 'date', and 'sales'.
        valid_df : DataFrame
            Validation data including 'id', 'date', 'sales', and 'predicted'.

    Returns:
        overall_wrmsse : float
            The weighted RMSSE computed across series.
    """
    series_ids = valid_df['id'].unique()
    weighted_errors = []
    weights = []

    for sid in series_ids:
        # Get training sales history for this series.
        train_sales = train_df[train_df['id'] == sid].sort_values('date')['sales'].values
        # Get actual validation sales.
        valid_sales = valid_df[valid_df['id'] == sid].sort_values('date')['sales'].values
        # Get predictions for this series.
        pred_sales = valid_df[valid_df['id'] == sid].sort_values('date')['predicted'].values

        # Compute the scaling factor: RMS of the differences in training sales.
        if len(train_sales) > 1:
            scale = np.sqrt(np.mean(np.diff(train_sales) ** 2))
        else:
            scale = 1.0

        # Compute RMSE for the validation period for this series.
        rmse = np.sqrt(np.mean((valid_sales - pred_sales) ** 2))

        # Calculate the series-specific RMSSE.
        series_error = rmse / scale if scale > 0 else rmse

        # Weight for the series: Here we use the sum of training sales.
        series_weight = train_sales.sum()

        weighted_errors.append(series_error ** 2 * series_weight)
        weights.append(series_weight)

    overall_wrmsse = np.sqrt(np.sum(weighted_errors) / np.sum(weights))
    return overall_wrmsse

# Compute the simplified WRMSSE score.
wrmsse_score = simple_wrmsse(train_df, valid_df_with_preds)
print("Simplified WRMSSE score for Tweedie model:", wrmsse_score)